In [1]:
# Célula 1: Importar bibliotecas e carregar os dados (usando nosso método aperfeiçoado)
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

# --- Carregando os Dados ---
path_inep = '../data/raw/MICRODADOS_ED_SUP_IES_2023.CSV'
path_cnpq = '../data/raw/Relatorio_de_dados_abertos_CNPq.csv'

df_inep = pd.read_csv(path_inep, sep=';', encoding='latin-1')

dtype_cnpq = {0: 'str', 7: 'str', 23: 'str', 24: 'str'}
df_cnpq = pd.read_csv(path_cnpq, sep=';', encoding='latin-1', skiprows=5, dtype=dtype_cnpq)

print("Datasets carregados com sucesso!")

Datasets carregados com sucesso!


In [2]:
# Célula 2: Seleção de Colunas Relevantes
# Selecionar apenas as colunas que realmente interessam para a análise.

# Colunas de interesse no dataset do INEP
colunas_inep_selecionadas = [
    'CO_IES', 'NO_IES', 'SG_IES', 'NO_MUNICIPIO_IES', 'SG_UF_IES',
    'TP_CATEGORIA_ADMINISTRATIVA', 'QT_DOC_EX_DOUT', 'QT_DOC_EX_MEST'
]

# Colunas de interesse no dataset do CNPq (agora com os nomes corretos)
colunas_cnpq_selecionadas = [
    'Modalidade',
    'Instituição Destino',
    'Sigla Instituição Destino',
    'Grande Área',
    'Área'
]

df_inep_selecionado = df_inep[colunas_inep_selecionadas].copy()
# Usamos .get() para evitar KeyErrors se alguma coluna não existir, embora agora deva existir.
df_cnpq_selecionado = df_cnpq[colunas_cnpq_selecionadas].copy()

print("Colunas selecionadas. Formato dos DataFrames:")
print("INEP:", df_inep_selecionado.shape)
print("CNPq:", df_cnpq_selecionado.shape)

df_cnpq_selecionado.head()

Colunas selecionadas. Formato dos DataFrames:
INEP: (2580, 8)
CNPq: (115699, 5)


,Modalidade,Instituição Destino,Sigla Instituição Destino,Grande Área,Área
0,IC - Iniciação Científica,Universidade Federal do Rio Grande do Norte,UFRN,"Lingüística, Letras e Artes",Artes
1,IC - Iniciação Científica,Universidade Federal do Rio Grande do Norte,UFRN,"Lingüística, Letras e Artes",Artes
2,IC - Iniciação Científica,Universidade Federal do Rio Grande do Norte,UFRN,"Lingüística, Letras e Artes",Artes
3,IC - Iniciação Científica,Universidade Federal do Rio Grande do Norte,UFRN,"Lingüística, Letras e Artes",Artes
4,IT - Iniciação Tecnológica,"Instituto Federal de Educação, Ciência e Tecno...",IFSULDEMINAS,Engenharias,Engenharia Sanitária


In [3]:
# Célula 3: Padronização dos Nomes das Instituições
# Criar uma função para "limpar" os nomes das IES, deixando-os em um formato padrão.
# Isso aumenta muito a chance de os nomes combinarem entre os dois arquivos.
# A função irá converter para minúsculas, remover acentos e caracteres especiais.

# Primeiro, instala uma biblioteca útil para remover acentos
!pip install unidecode

from unidecode import unidecode

def padronizar_nome(nome):
    if not isinstance(nome, str):
        return ''
    # Converte para minúsculas
    nome = nome.lower()
    # Remove acentos
    nome = unidecode(nome)
    # Remove pontuações e caracteres especiais comuns
    nome = nome.replace('.', '').replace('-', ' ').replace('  ', ' ')
    # Remove espaços no início e no fim
    return nome.strip()

# Aplicando a função para criar uma nova coluna padronizada em cada DataFrame
df_inep_selecionado['nome_ies_padronizado'] = df_inep_selecionado['NO_IES'].apply(padronizar_nome)
df_cnpq_selecionado['nome_ies_padronizado'] = df_cnpq_selecionado['Instituição Destino'].apply(padronizar_nome)

print("Nomes das instituições padronizados com sucesso.")
# Visualizando o resultado da padronização
df_inep_selecionado[['NO_IES', 'nome_ies_padronizado']].head()

Nomes das instituições padronizados com sucesso.


,NO_IES,nome_ies_padronizado
0,UNIVERSIDADE FEDERAL DE MATO GROSSO,universidade federal de mato grosso
1,UNIVERSIDADE DE BRASÍLIA,universidade de brasilia
2,UNIVERSIDADE FEDERAL DE SERGIPE,universidade federal de sergipe
3,UNIVERSIDADE FEDERAL DO AMAZONAS,universidade federal do amazonas
4,UNIVERSIDADE FEDERAL DO PIAUÍ,universidade federal do piaui


In [4]:
# Célula 4: Agregando os Dados do CNPq
# O dataset do CNPq tem uma linha para cada bolsa. Precisa resumi-lo para ter
# uma linha para cada instituição, com o total de bolsas.

print("Agregando os dados do CNPq por instituição...")

# Contando o total de bolsas por instituição
df_cnpq_agregado = df_cnpq_selecionado.groupby('nome_ies_padronizado').agg(
    total_bolsas_cnpq=('Modalidade', 'size') # 'size' é eficiente para contar linhas
).reset_index()


# Contar o total por tipo de modalidade,
# pois isso será útil na análise.
df_cnpq_pivot = df_cnpq_selecionado.pivot_table(
    index='nome_ies_padronizado',
    columns='Modalidade',
    values='Instituição Destino', # Qualquer coluna serve para contar
    aggfunc='size',
    fill_value=0
)

# Juntando a contagem total com a contagem por modalidade
df_cnpq_final = pd.merge(df_cnpq_agregado, df_cnpq_pivot, on='nome_ies_padronizado', how='left')

print("Dados do CNPq agregados com sucesso!")
df_cnpq_final.head()

Agregando os dados do CNPq por instituição...
Dados do CNPq agregados com sucesso!


,nome_ies_padronizado,total_bolsas_cnpq,ADC - Apoio à Difusão do Conhecimento,AED - Auxílio a Editoração e Publicação,APQ - Auxílio a Pesquisa,ARC - Promoção de Eventos Científicos,AT - Apoio Técnico a Pesquisa,ATP - Apoio Técnico em Extensão no País,DCR - Desenvolvimento Cientifico Regional,DES - Desenvolvimento Tecnológico e Inovação,DT - Produtividade Desen. Tec. e Extensão,DTC - Desenvolvimento Tecnológico em TICs,DTI - Desenvolvimento Tecnológico Industrial,EPI - Bolsa de Treinamento - Programa,EV - Especialista Visitante,EXP - Extensão no País,GD - Doutorado,GDE - Doutorado no Exterior,GM - Mestrado,IC - Iniciação Científica,ICJ - Iniciação Científica Júnior,IEX - Iniciação ao Extensionismo,IT - Iniciação Tecnológica,ITC - Iniciação Tecnológica em TICs,ITI - Iniciação Tecnológica e Industrial,PAV - Bolsa Pesquisador-Avaliador,PCI - Programa de Capacitação Institucional,PDE - Pós-doutorado no Exterior,PDI - Pos-doutorado Empresarial,PDJ - Pós-doutorado Júnior,PDS - Pós-doutorado Sênior,PQ - Produtividade em Pesquisa,PV - Pesquisador Visitante,SET - Fixação de Recursos Humanos,SWE - Doutorado Sanduíche no Exterior,SWP - Doutorado-Sanduiche no Pais
0,aarhus university,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,academia brasileira de ciencias,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,academisch centrum tandheelkunde amsterdam,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0
3,"acao educativa, assessoria, pesquisa e informacao",1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,acosta aerospace,5,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0


In [5]:
# Célula 5: Unificando (Merge) os DataFrames INEP e CNPq

print("Iniciando a unificação dos dataframes...")

# Usando um 'left' merge para manter todas as instituições do Censo (df_inep) como base,
# mesmo que elas não tenham bolsas no CNPq.
df_unificado = pd.merge(
    df_inep_selecionado,
    df_cnpq_final,
    on='nome_ies_padronizado',
    how='left'
)

# Após o merge, as instituições do INEP que não encontraram correspondência no CNPq
# terão valores 'NaN' (nulos) nas colunas de bolsas. Vamos preencher com 0.
colunas_bolsas = list(df_cnpq_final.columns)
colunas_bolsas.remove('nome_ies_padronizado')

for coluna in colunas_bolsas:
    # Em vez de usar fillna(inplace=True), reatribuímos o resultado.
    df_unificado[coluna] = df_unificado[coluna].fillna(0)

    # Convertendo colunas de contagem para inteiro
    df_unificado[coluna] = df_unificado[coluna].astype(int)

print("Unificação concluída com sucesso e sem warnings!")
print("Formato do DataFrame final:", df_unificado.shape)

# Verificando o resultado para algumas universidades conhecidas
df_unificado[df_unificado['SG_IES'].isin(['UFPA', 'USP', 'UFRJ', 'UNICAMP'])].head()

Iniciando a unificação dos dataframes...
Unificação concluída com sucesso e sem warnings!
Formato do DataFrame final: (2580, 44)


,CO_IES,NO_IES,SG_IES,NO_MUNICIPIO_IES,SG_UF_IES,TP_CATEGORIA_ADMINISTRATIVA,QT_DOC_EX_DOUT,QT_DOC_EX_MEST,nome_ies_padronizado,total_bolsas_cnpq,ADC - Apoio à Difusão do Conhecimento,AED - Auxílio a Editoração e Publicação,APQ - Auxílio a Pesquisa,ARC - Promoção de Eventos Científicos,AT - Apoio Técnico a Pesquisa,ATP - Apoio Técnico em Extensão no País,DCR - Desenvolvimento Cientifico Regional,DES - Desenvolvimento Tecnológico e Inovação,DT - Produtividade Desen. Tec. e Extensão,DTC - Desenvolvimento Tecnológico em TICs,DTI - Desenvolvimento Tecnológico Industrial,EPI - Bolsa de Treinamento - Programa,EV - Especialista Visitante,EXP - Extensão no País,GD - Doutorado,GDE - Doutorado no Exterior,GM - Mestrado,IC - Iniciação Científica,ICJ - Iniciação Científica Júnior,IEX - Iniciação ao Extensionismo,IT - Iniciação Tecnológica,ITC - Iniciação Tecnológica em TICs,ITI - Iniciação Tecnológica e Industrial,PAV - Bolsa Pesquisador-Avaliador,PCI - Programa de Capacitação Institucional,PDE - Pós-doutorado no Exterior,PDI - Pos-doutorado Empresarial,PDJ - Pós-doutorado Júnior,PDS - Pós-doutorado Sênior,PQ - Produtividade em Pesquisa,PV - Pesquisador Visitante,SET - Fixação de Recursos Humanos,SWE - Doutorado Sanduíche no Exterior,SWP - Doutorado-Sanduiche no Pais
37,54,UNIVERSIDADE ESTADUAL DE CAMPINAS,UNICAMP,Campinas,SP,2,1768,19,universidade estadual de campinas,2844,1,5,14,0,20,8,0,0,7,2,73,0,3,1,483,0,277,805,649,0,48,0,27,0,0,0,2,37,6,368,1,7,0,0
38,55,UNIVERSIDADE DE SÃO PAULO,USP,São Paulo,SP,2,5546,51,universidade de sao paulo,3950,2,10,31,0,52,5,0,0,9,1,212,0,1,4,683,0,472,1366,256,1,88,2,30,1,0,0,0,46,21,648,4,4,0,1
254,569,UNIVERSIDADE FEDERAL DO PARÁ,UFPA,Belém,PA,1,2277,390,universidade federal do para,1221,6,2,22,0,8,22,3,0,15,0,25,0,1,3,117,0,149,482,38,3,10,3,32,0,0,0,0,13,2,255,1,9,0,0
271,586,UNIVERSIDADE FEDERAL DO RIO DE JANEIRO,UFRJ,Rio de Janeiro,RJ,1,3950,663,universidade federal do rio de janeiro,3715,2,3,26,2,30,15,0,0,13,2,95,0,6,8,593,0,308,1256,147,0,103,9,77,0,0,0,0,50,9,950,1,9,0,1


In [6]:
# Célula 6: Salvando o Resultado Final
# Como boa prática, salvar nosso dataframe limpo e unificado.
# Vai ser o ponto de partida para a próxima etapa de Análise e Visualização.

caminho_saida = '../data/processed/censo_cnpq_unificado_2023.csv'
df_unificado.to_csv(caminho_saida, index=False)

print(f"DataFrame final salvo com sucesso em: {caminho_saida}")

DataFrame final salvo com sucesso em: ../data/processed/censo_cnpq_unificado_2023.csv
